In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

def dia():
    from datetime import date, timedelta
    dia = date.today() + timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df
    
def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

## Instalando as Dependências do Ubuntu

In [2]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list <<'EOF'
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500

# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300

# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# # Install selenium
# pip install selenium

## Configuração do Web-Driver

In [3]:
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.headless = True

from selenium.webdriver.chrome.service import Service
wd_Chrome = webdriver.Chrome("/usr/bin/chromedriver", options=options)

## Iniciando a Raspagem de Dados

In [4]:
# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com/")

In [5]:
# Fechando Botão de Cookies
try:
    button_cookies = wd_Chrome.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
    button_cookies.click()
except:
    pass

# Selecionando o Dia de Amanhã
wd_Chrome.find_element(By.CSS_SELECTOR,'button.calendar__navigation--tomorrow').click()
time.sleep(3)

# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

jogo = {'Date':[],'Time':[],'Country':[],'League':[],'Home':[],'Away':[],'Odd_H':[],'Odd_D':[],'Odd_A':[],'Odd_Over15':[],'Odd_Under15':[],'Odd_Over25':[],'Odd_Under25':[],'Odd_BTTS_Yes':[],'Odd_BTTS_No':[]}

In [6]:
# Pegando as Informacoes Básicas do Jogo
for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/match-summary')
    
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
    except:
        pass

# Match Odds
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/1x2-odds/full-time')
        time.sleep(2)
        
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                Odd_H = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odd_D = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text) 
                Odd_A = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text)
    except:
        pass

# Over e Under 2.5
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/over-under/full-time')
        time.sleep(2)
        
        linhas_over_under = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table.oddsCell__odds')
        for i in linhas_over_under:
            linhas = i.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
            for linha in linhas:
                try:
                    qtd_gols = linha.find_element(By.CSS_SELECTOR,'span.oddsCell__noOddsCell').text.replace('.','')
                    if qtd_gols == '05':
                        Odd_Over05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                    if qtd_gols == '15':
                        Odd_Over15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                        #print(Odd_Under15)
                    if qtd_gols == '25':
                        Odd_Over25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                except:
                    pass
    except:
        pass

# Ambas Marcam (BTTS)
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/both-teams-to-score/full-time')
        time.sleep(2)
    
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                BTTS_Yes = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                BTTS_No = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
    except:
        pass    
    
    try:
        print(Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,BTTS_Yes,BTTS_No) 

        jogo['Date'].append(Date)
        jogo['Time'].append(Time)
        jogo['Country'].append(Country)
        jogo['League'].append(League)
        jogo['Home'].append(Home)
        jogo['Away'].append(Away)
        jogo['Odd_H'].append(Odd_H)
        jogo['Odd_D'].append(Odd_D)
        jogo['Odd_A'].append(Odd_A)
        jogo['Odd_Over15'].append(Odd_Over15)
        jogo['Odd_Under15'].append(Odd_Under15)
        jogo['Odd_Over25'].append(Odd_Over25)
        jogo['Odd_Under25'].append(Odd_Under25)
        jogo['Odd_BTTS_Yes'].append(BTTS_Yes)
        jogo['Odd_BTTS_No'].append(BTTS_No)

    except:
        pass

 72%|███████▏  | 138/193 [19:30<08:53,  9.70s/it]

09.07.2023 12:00 SWEDEN SUPERETTAN - ROUND 13 Skovde AIK GAIS 5.0 4.2 1.6 1.22 4.2 1.72 2.1 1.78 2.0


 72%|███████▏  | 139/193 [19:38<08:28,  9.42s/it]

09.07.2023 08:00 SWEDEN ALLSVENSKAN WOMEN - ROUND 17 IF Brommapojkarna W Norrkoping W 3.6 3.75 1.83 1.22 4.2 1.72 2.1 1.65 2.2


 73%|███████▎  | 140/193 [19:47<08:07,  9.19s/it]

...

09.07.2023 10:00 ZIMBABWE PREMIER SOCCER LEAGUE - ROUND 14 Hwange ZPC Kariba 1.72 3.15 4.75 1.53 2.18 2.62 1.39 2.65 1.45


 99%|█████████▉| 192/193 [26:32<00:07,  7.25s/it]

09.07.2023 10:00 ZIMBABWE PREMIER SOCCER LEAGUE - ROUND 14 Sheasham Ngezi Platinum 1.72 3.15 4.75 1.53 2.18 2.62 1.39 2.65 1.45


100%|██████████| 193/193 [26:39<00:00,  8.29s/it]

09.07.2023 10:00 ZIMBABWE PREMIER SOCCER LEAGUE - ROUND 14 Simba Bhora Manica 1.72 3.15 4.75 1.53 2.18 2.62 1.39 2.65 1.45


## Gerando DataFrame com os Jogos do Dia

In [7]:
df = pd.DataFrame(jogo)
df = drop_reset_index(df)
df

,Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,Odd_BTTS_Yes,Odd_BTTS_No
1,09.07.2023,18:00,NORTH & CENTRAL AMERICA,GOLD CUP - PLAY OFFS - QUARTER-FINALS,Guatemala,Jamaica,5.50,3.60,1.66,1.42,2.85,2.25,1.62,2.20,1.63
2,09.07.2023,20:30,NORTH & CENTRAL AMERICA,GOLD CUP - PLAY OFFS - QUARTER-FINALS,USA,Canada,1.68,3.75,5.00,1.30,3.40,1.95,1.83,1.90,1.83
3,09.07.2023,10:00,AFRICA,COSAFA CUP,Zambia,Comoros,1.68,3.10,5.50,1.57,2.25,2.88,1.40,2.55,1.45
4,09.07.2023,13:00,AFRICA,COSAFA CUP,Malawi,Seychelles,1.45,3.75,7.00,1.40,2.75,2.20,1.60,2.40,1.50
5,09.07.2023,15:30,ARGENTINA,LIGA PROFESIONAL - ROUND 24,Instituto,Tigre,2.25,3.15,3.30,1.43,2.75,2.32,1.58,1.97,1.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,09.07.2023,10:00,ZIMBABWE,PREMIER SOCCER LEAGUE - ROUND 14,GreenFuel,Chicken Inn,1.04,11.00,31.00,1.02,10.00,1.17,4.50,1.95,1.75
190,09.07.2023,10:00,ZIMBABWE,PREMIER SOCCER LEAGUE - ROUND 14,Highlanders,Triangle,1.72,3.15,4.75,1.53,2.18,2.62,1.39,2.65,1.45
191,09.07.2023,10:00,ZIMBABWE,PREMIER SOCCER LEAGUE - ROUND 14,Hwange,ZPC Kariba,1.72,3.15,4.75,1.53,2.18,2.62,1.39,2.65,1.45
192,09.07.2023,10:00,ZIMBABWE,PREMIER SOCCER LEAGUE - ROUND 14,Sheasham,Ngezi Platinum,1.72,3.15,4.75,1.53,2.18,2.62,1.39,2.65,1.45


In [8]:
df = df.rename(columns={'Odd_H': 'FT_Odd_H',
                        'Odd_D': 'FT_Odd_D',
                        'Odd_A': 'FT_Odd_A',
                        'Odd_Over15': 'FT_Odd_Over15',
                        'Odd_Under15': 'FT_Odd_Under15',
                        'Odd_Over25': 'FT_Odd_Over25',
                        'Odd_Under25': 'FT_Odd_Under25',
                        'Odd_BTTS_Yes': 'FT_Odd_BTTS_Yes',
                        'Odd_BTTS_No': 'FT_Odd_BTTS_No',
                        })

In [9]:
# Gerando os dados para excel
from datetime import date, datetime, timedelta
data = datetime.today() + timedelta(1)
amanha = data.strftime('%Y-%m-%d')
nome = 'Jogos_do_Dia_FlashScore.xlsx'

df.to_excel(f'G:\\Meu Drive\\Programacao\\futPythonTrader\\Jogos do dia\\{amanha}_{nome}', index = False)